In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pickle
import os
import tensorflow as tf
from tensorflow.contrib import rnn
import sys
from scipy import sparse

target_columns = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
           'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
           'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
           'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
           'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
           'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
           'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
           'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [15]:
total_no_interactions = 0
array_ordered = np.arange(1, 25)
def evaluate_sample(predictions, y_true, k):
    global total_no_interactions
    global array_ordered
    sorted_pred, sorted_y = zip(*sorted(zip(predictions, y_true[0]), reverse=True )) #TODO: Discard the products that were already part of the portfolio in the last step
    #Recall at k
    true_pos_k = sum(sorted_y[0:k])
    num_pos = sum(y_true[0])
    recall_user = (true_pos_k/float(num_pos))
    #Map at k
    precisions = sorted_y/array_ordered
    sum_precisions = np.sum(precisions[:k])
    if num_pos == 0:
        map_k = 0
        recall_user = 0
        total_no_interactions += 1
    else:
        map_k = sum_precisions/float(min(num_pos, k))
        #map_k = sum_precisions/float(k)

    #print('sorted_pred: ' + str(sorted_pred))
    #print('sorted_y: ' + str(sorted_y))
    #print('true_pos_k: ' + str(true_pos_k))
    #print('num_pos: ' + str(num_pos))
    #print('recall_user: ' + str(recall_user))
    #print('precisions: ' + str(precisions))
    #print('sum_precisions: ' + str(sum_precisions))
    #print('map_k: ' + str(map_k))
    


    return recall_user, true_pos_k, num_pos, map_k

In [ ]:
representation = 4
max_interactions = 5
padding = 'right'
aux_features_length = '0'
with open('pickles/X_train_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length +  '.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('pickles/X_test_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    X_test = pickle.load(handle)
with open('pickles/Y_train_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    Y_train = pickle.load(handle)


In [ ]:
num_features = X_train[0].shape[0] * X_train[0].shape[1]
num_output = Y_train[0].shape[1]
X_train_no_sparse = np.zeros((len(X_train), num_features), dtype=np.int8)
Y_train_no_sparse = np.zeros((len(Y_train), num_output), dtype=np.int8)
for i in range(len(X_train)):
    X_train_no_sparse[i] = X_train[i].toarray().reshape(num_features)
    Y_train_no_sparse[i] = Y_train[i].toarray().reshape(num_output)
X_train_no_sparse = sparse.csr_matrix(X_train_no_sparse, dtype=np.int8)
Y_train_no_sparse = sparse.csr_matrix(Y_train_no_sparse, dtype=np.int8)


In [ ]:
X_train_no_sparse

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=25)
clf.fit(X_train_no_sparse.toarray(), Y_train_no_sparse.toarray())


In [ ]:
Y_train_no_sparse.toarray()

# Evaluation

In [ ]:
with open('pickles/X_local_test_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    X_local_test = pickle.load(handle)
with open('pickles/Y_local_test_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    Y_local_test = pickle.load(handle)
X_local_test = np.array(X_local_test)
Y_local_test = np.array(Y_local_test)

In [ ]:
X_test_no_sparse.shape

In [ ]:
X_test_no_sparse = np.zeros((len(X_local_test), num_features), dtype=np.int8)
Y_test_no_sparse = np.zeros((len(Y_local_test), num_output), dtype=np.int8)
for i in range(len(X_local_test)):
    X_test_no_sparse[i] = X_local_test[i].reshape(num_features)
    Y_test_no_sparse[i] = Y_local_test[i].reshape(num_output)
X_test_no_sparse = sparse.csr_matrix(X_test_no_sparse, dtype=np.int8)
Y_test_no_sparse = sparse.csr_matrix(Y_test_no_sparse, dtype=np.int8)

In [ ]:
pred_local_test = clf.predict_proba(X_test_no_sparse)

In [ ]:
Y_train_no_sparse[0]

In [ ]:
print(pred_local_test[4])

In [ ]:
X_local_test = np.array(X_local_test)
Y_local_test = np.array(Y_local_test)
k = 7
with open('pickles/indices_interactions_local_test.pickle', 'rb') as handle:
    indices_interactions = pickle.load(handle)
recall_users = []
total_true_pos_k = 0
total_pos = 0
with open('pickles/indices_interactions_local_test.pickle', 'rb') as handle:
    indices_interactions = pickle.load(handle)

In [ ]:
for i in range(len(pred_local_test)):
    if i in indices_interactions:
        #sorted_pred, sorted_y = zip(*sorted(zip(pred_local_test[i], X_local_test[i]), reverse=True ))
        recall_user, true_pos_k, num_pos, map_k = evaluate_sample(pred_local_test[i], Y_local_test[i], k)
        recall_users.append(recall_user)
        total_true_pos_k += true_pos_k
        total_pos += num_pos

In [ ]:
print('Results local test:')
print('Total users evaluated: ' + str(len(recall_users)))
print('Total True positives at k: :' + str(total_true_pos_k))
print('Total True positives: ' + str(total_pos))
recall_k = total_true_pos_k/ float(total_pos)
print('Total Recall at ' + str(k) + ': ' + str(recall_k))
print('Mean recall at ' + str(k) + ' by user: ' + str(np.mean(recall_users)))
print('Mean map at ' + str(k) + ' by user: ' + str(map_k))

# Frequency Baseline

In [ ]:
with open('pickles/sorted_freq.pickle', 'rb') as handle:
    sorted_freq = pickle.load(handle)
with open('pickles/sorted_prods_total.pickle', 'rb') as handle:
    sorted_prods_total = pickle.load(handle)

In [ ]:
target_freq = np.zeros(24)
for i in range(len(sorted_freq)):
    idx_product = target_columns.index(sorted_prods_total[i])
    prob_product = sorted_freq[i]
    target_freq[idx_product] = prob_product


In [ ]:
recall_users = []
map_k_users = []
total_true_pos_k = 0
total_pos = 0
k = 7
for i in range(len(Y_local_test)):
    recall_user, true_pos_k, num_pos, map_k = evaluate_sample(target_freq, Y_local_test[i], k)
    map_k_users.append(map_k)
    recall_users.append(recall_user)
    total_true_pos_k += true_pos_k
    total_pos += num_pos
    #break

In [ ]:
target_freq

In [ ]:
print('Results local test BASELINE:')
print('Total users evaluated: ' + str(len(recall_users)))
print('Total True positives at k: :' + str(total_true_pos_k))
print('Total True positives: ' + str(total_pos))
recall_k = total_true_pos_k/ float(total_pos)
print('Total Recall at ' + str(k) + ': ' + str(recall_k))
print('Mean recall at ' + str(k) + ' by user: ' + str(np.mean(recall_users)))
print('Mean map at ' + str(k) + ' by user: ' + str(np.mean(map_k_users)))

# Using one vs Classifier

In [45]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA


In [46]:
representation = 4
max_interactions = 5
padding = 'right'
aux_features_length = '0'
with open('pickles/X_train_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length +  '.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('pickles/X_test_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    X_test = pickle.load(handle)
with open('pickles/Y_train_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    Y_train = pickle.load(handle)


In [47]:
num_features = X_train[0].shape[0] * X_train[0].shape[1]
num_output = Y_train[0].shape[1]
X_train_no_sparse = np.zeros((len(X_train), num_features), dtype=np.int8)
Y_train_no_sparse = np.zeros((len(Y_train), num_output), dtype=np.int8)
for i in range(len(X_train)):
    X_train_no_sparse[i] = X_train[i].toarray().reshape(num_features)
    Y_train_no_sparse[i] = Y_train[i].toarray().reshape(num_output)
X_train_no_sparse = sparse.csr_matrix(X_train_no_sparse, dtype=np.int8)
Y_train_no_sparse = sparse.csr_matrix(Y_train_no_sparse, dtype=np.int8)


In [75]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
classif = OneVsRestClassifier(LogisticRegression(penalty='l2', tol=0.0001, C=1.0, random_state=17, max_iter=100,  verbose=0))
#classif = OneVsRestClassifier(LogisticRegression(penalty='l2', tol=0.0001, C=10.0, random_state=17, max_iter=100,  verbose=0))
#classif = OneVsRestClassifier(LogisticRegression(penalty='l2', tol=0.0001, C=0.1, random_state=17, max_iter=100,  verbose=0))
#classif = OneVsRestClassifier(ExtraTreesClassifier(n_estimators=40, max_depth=30, random_state=17))
#classif = OneVsRestClassifier(SVC(kernel='rbf', probability=True))
classif.fit(X_train_no_sparse, Y_train_no_sparse)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\multiclass.py:70: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=150, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [49]:
with open('pickles/X_local_test_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    X_local_test = pickle.load(handle)
with open('pickles/Y_local_test_rep' + str(representation) + '_' + str(max_interactions) + '_' + padding + '_' + aux_features_length + '.pickle', 'rb') as handle:
    Y_local_test = pickle.load(handle)
X_local_test = np.array(X_local_test)
Y_local_test = np.array(Y_local_test)

In [50]:
X_test_no_sparse = np.zeros((len(X_local_test), num_features), dtype=np.int8)
Y_test_no_sparse = np.zeros((len(Y_local_test), num_output), dtype=np.int8)
for i in range(len(X_local_test)):
    X_test_no_sparse[i] = X_local_test[i].reshape(num_features)
    Y_test_no_sparse[i] = Y_local_test[i].reshape(num_output)
X_test_no_sparse = sparse.csr_matrix(X_test_no_sparse, dtype=np.int8)
Y_test_no_sparse = sparse.csr_matrix(Y_test_no_sparse, dtype=np.int8)

In [76]:
probs = classif.predict_proba(X_test_no_sparse)

In [52]:
probs.shape

(23093, 24)

In [77]:
recalls_model = []
recalls_baseline = []
maps_model = []
maps_baseline = []
ks = [1,2,3,4,5,6,7,8,9]
for k in ks:
    recall_users = []
    map_k_users = []
    total_true_pos_k = 0
    total_pos = 0




    for i in range(len(probs)):  
        #sorted_pred, sorted_y = zip(*sorted(zip(pred_local_test[i], Y_local_test[i]), reverse=True )) #TODO: Discard the products that were already part of the portfolio in the last step
        recall_user, true_pos_k, num_pos, map_k = evaluate_sample(probs[i], Y_test_no_sparse[i].toarray(), k)
        map_k_users.append(map_k)
        recall_users.append(recall_user)
        total_true_pos_k += true_pos_k
        total_pos += num_pos
        if i % 100000 == 0:
            print(i)
                
    print('-------------------')            
    print('Results local test for k = ' + str(k) + ':')
    print('Total users evaluated: ' + str(len(recall_users)))
    print('Total True positives at k: :' + str(total_true_pos_k))
    print('Total True positives: ' + str(total_pos))
    recall_k = total_true_pos_k/ float(total_pos)
    print('Total Recall at ' + str(k) + ': ' + str(recall_k))
    print('Mean recall at ' + str(k) + ' by user: ' + str(np.mean(recall_users)))
    print('Mean map at ' + str(k) + ' by user: ' + str(np.mean(map_k_users)))
    #print('Max map at ' + str(k) + ' by user: ' + str(np.max(map_k_users)))   
    #print('Min map at ' + str(k) + ' by user: ' + str(np.min(map_k_users))) 
    print('Total no interactions: ' + str(total_no_interactions))
    recalls_model.append(np.mean(recall_users))
    maps_model.append(np.mean(map_k_users))

0
-------------------
Results local test for k = 1:
Total users evaluated: 23093
Total True positives at k: :19263
Total True positives: 29573
Total Recall at 1: 0.651371183174
Mean recall at 1 by user: 0.728583697802
Mean map at 1 by user: 0.834148876283
Total no interactions: 0
0
-------------------
Results local test for k = 2:
Total users evaluated: 23093
Total True positives at k: :25074
Total True positives: 29573
Total Recall at 2: 0.847867987691
Mean recall at 2 by user: 0.870820161954
Mean map at 2 by user: 0.808470099164
Total no interactions: 0
0
-------------------
Results local test for k = 3:
Total users evaluated: 23093
Total True positives at k: :26867
Total True positives: 29573
Total Recall at 3: 0.908497616069
Mean recall at 3 by user: 0.916217757185
Mean map at 3 by user: 0.815388982712
Total no interactions: 0
0
-------------------
Results local test for k = 4:
Total users evaluated: 23093
Total True positives at k: :27691
Total True positives: 29573
Total Recall a

In [20]:
Y_test_no_sparse[0].toarray()[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0], dtype=int8)